In [ ]:
import os
import tensorflow as tf
import keras
# MLP for Pima Indians Dataset Serialize to JSON and HDF5

print(tf.version.VERSION)

In [ ]:
#Load the data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
len(train_images)

In [ ]:

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

#Normalize the data
train_images = train_images[:1000].astype('float32') / 255
test_images = test_images[:1000].astype('float32') / 255


In [ ]:
# Reshape the data
train_images = train_images.reshape((train_images.shape[0],  28, 28,1))
test_images = test_images.reshape((test_images.shape[0],  28, 28,1))

In [ ]:
len(train_images)

# Define a simple sequential model

In [ ]:
def create_model():
    model = tf.keras.Sequential()
    # Must define the input shape in the first layer of the neural network
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    #Compiling the model
    model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
    
    return model

In [ ]:
# Create a basic model instance
model = create_model()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
# Take a look at the model summary
model.summary()

In [ ]:
model.fit(train_images, 
          train_labels,  
          batch_size=64,
          epochs=100
          ,
          validation_data=(test_images,test_labels))

In [ ]:
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
from keras.models import model_from_json

In [ ]:
# serialize model to json
json_model = model.to_json()
#json_model

In [ ]:

#save the model architecture to JSON file
with open('fashionmnist_model.json', 'w') as json_file:
    json_file.write(json_model)


In [ ]:
#saving the weights of the model
model.save_weights('FashionMNIST_weights.h5')

In [ ]:

from keras.initializers import glorot_uniform

# Creating a new model from the saved JSON file
# reda the model from the JSOn file
with open('fashionmnist_model.json', 'r') as json_file:
    json_savedModel= json_file.read()
#json_savedModel
#load the model architecture 
model_j = tf.keras.models.model_from_json(json_savedModel)
model_j.summary()

In [ ]:
model_j.load_weights('FashionMNIST_weights.h5')

In [ ]:
#Compiling the model
model_j.compile(loss='sparse_categorical_crossentropy',
         optimizer='SGD',
         metrics=['accuracy'])

In [ ]:
loss,acc = model_j.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
#saving the model to a YAML file
yaml_model= model.to_yaml()
#yaml_model

In [ ]:
# writing the yaml model to the yaml file
with open('fashionmnist_yamlmodel.yaml', 'w') as yaml_file:
    yaml_file.write(yaml_model)

In [ ]:
#Read  the model architecture to YAML file
with open('fashionmnist_yamlmodel.yaml', 'r') as yaml_file:
    yaml_savedModel=yaml_file.read()

In [ ]:
# Load the saved Yaml model
from keras.models import model_from_yaml
model_y= tf.keras.models.model_from_yaml(yaml_savedModel)
model_y.summary()

In [ ]:
#saving the smodel's architecture, weights, and training configuration in a single file/folder. 
#This allows you to export a model so it can be used without access to the original Python code
#Since the optimizer-state is recovered, you can resume training from exactly where you left off.
model.save('fashionmnist.h5')

In [ ]:
#loading the model from the Yaml file

In [ ]:
# loading the model from the H5 file
model_h5 = tf.keras.models.load_model('fashionmnist.h5')
model_h5.summary()

In [ ]:
loss,acc = model_h5.evaluate(test_images,  test_labels, verbose=2)

In [12]:
def create_model():
    model = tf.keras.Sequential()
    # Must define the input shape in the first layer of the neural network
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    #Compiling the model
    model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
    
    return model

In [13]:
model_ckpt= create_model()

In [14]:
checkpoint_path = "train_ckpt/cp.ckpt"


In [15]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_best_only=True, save_weights_only=True, verbose=1)

In [16]:
# Train the model with the new callback
model_ckpt.fit(train_images, 
          train_labels,  
          batch_size=64,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.9825 - acc: 0.2708
Epoch 00001: val_loss improved from inf to 1.38169, saving model to train_ckpt/cp.ckpt
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.9589 - acc: 0.2770 - val_loss: 1.3817 - val_acc: 0.5780
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.3116 - acc: 0.5115
Epoch 00002: val_loss improved from 1.38169 to 1.02368, saving model to train_ckpt/cp.ckpt
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.3027 - acc: 0.5150 - val_loss: 1.0237 - val_acc: 0.6320
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.0952 - acc: 0.5740
Epoch 00003: val_loss improved from 1.02368 to 0.90785, saving model to train_ckpt/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.0864 - acc: 0.5800 - val_loss: 0.9078 - val_acc: 0.6520
Epoch 4/10
 960/1000 [=======

In [17]:
loss,acc = model_ckpt.evaluate(test_images,  test_labels, verbose=2)

1000/1000 - 0s - loss: 0.6148 - acc: 0.7800


In [18]:
# Create a basic model instance
model_ckpt2 = create_model()

In [19]:
model_ckpt2.load_weights(checkpoint_path)

In [20]:
loss,acc = model_ckpt2.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 1s - loss: 0.6124 - acc: 0.7610
Restored model, accuracy: 76.10%


In [21]:
loss,acc = model_ckpt2.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.6124 - acc: 0.7610
Restored model, accuracy: 76.10%


In [22]:
# Train the model with the new callback
model_ckpt2.fit(train_images, 
          train_labels,  
          batch_size=64,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.5791 - acc: 0.7927
Epoch 00001: val_loss improved from 0.61243 to 0.61077, saving model to train_ckpt/cp.ckpt
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.5883 - acc: 0.7900 - val_loss: 0.6108 - val_acc: 0.7710
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.5698 - acc: 0.7969
Epoch 00002: val_loss did not improve from 0.61077
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.5755 - acc: 0.7940 - val_loss: 0.6168 - val_acc: 0.7730
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.5573 - acc: 0.8021
Epoch 00003: val_loss improved from 0.61077 to 0.60367, saving model to train_ckpt/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.5532 - acc: 0.8050 - val_loss: 0.6037 - val_acc: 0.7800
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s -

In [23]:
loss,acc = model_ckpt2.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.5376 - acc: 0.8020
Restored model, accuracy: 80.20%


In [24]:
#get the latest checkpoint file
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [25]:
#Create a new model instance
model_latest_checkpoint = create_model()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)
# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.5365 - acc: 0.8000
Restored model, accuracy: 80.00%


In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "traiinig2/cp-{epoch:04d}.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)

In [ ]:
# Create a basic model instance
model_3 = create_model()

In [ ]:
# Train the model with the new callback
model_3.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

In [ ]:
!dir traiinig2


In [ ]:
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)


In [ ]:
# Create a new model instance
model_latest_checkpoint = create_model()

# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)

# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

## https://www.tensorflow.org/tutorials/keras/save_and_load#setup